In [1]:
import dotenv
dotenv.load_dotenv()

True

In [2]:
llm_config={"model": "gpt-3.5-turbo"}

In [3]:
task = '''
        Write a concise but engaging blogpost about
       DeepLearning.AI. Make sure the blogpost is
       within 100 words.
       '''


In [4]:
import autogen

writer = autogen.AssistantAgent(
    name="Writer",
    system_message="You are a writer. You write engaging and concise " 
        "blogpost (with title) on given topics. You must polish your "
        "writing based on the feedback you receive and give a refined "
        "version. Only return your final work without additional comments.",
    llm_config=llm_config,
)

In [5]:
reply = writer.generate_reply(messages=[{"content": task, "role": "user"}])

In [6]:
print(reply)

Title: Unleashing the Power of DeepLearning.AI

Dive into the world of AI with DeepLearning.AI! Founded by the renowned Andrew Ng, this platform offers top-notch courses on deep learning, neural networks, and machine learning. Whether you're a beginner or an advanced learner, there's something for everyone. With hands-on projects and expert-led videos, you'll sharpen your AI skills and stay ahead in the tech game. Join the community of learners globally, and unlock exciting opportunities in artificial intelligence. Embrace the future with DeepLearning.AI as your guide! Start your AI journey today.


#### Reflection

In [7]:
critic = autogen.AssistantAgent(
    name="Critic",
    is_termination_msg=lambda x: x.get("content", "").find("TERMINATE") >= 0,
    llm_config=llm_config,
    system_message="You are a critic. You review the work of "
                "the writer and provide constructive "
                "feedback to help improve the quality of the content.",
)

In [8]:
res = critic.initiate_chat(
    recipient=writer,
    message=task,
    max_turns=2,
    summary_method="last_msg"
)

Critic (to Writer):


        Write a concise but engaging blogpost about
       DeepLearning.AI. Make sure the blogpost is
       within 100 words.
       

--------------------------------------------------------------------------------
Writer (to Critic):

Title: Unleashing the Power of DeepLearning.AI

Dive into the world of AI with DeepLearning.AI! Founded by the renowned Andrew Ng, this platform offers top-notch courses on deep learning, neural networks, and machine learning. Whether you're a beginner or an advanced learner, there's something for everyone. With hands-on projects and expert-led videos, you'll sharpen your AI skills and stay ahead in the tech game. Join the community of learners globally, and unlock exciting opportunities in artificial intelligence. Embrace the future with DeepLearning.AI as your guide! Start your AI journey today.

--------------------------------------------------------------------------------
Critic (to Writer):

This blogpost does a great job o

#### Nested chat

In [10]:
SEO_reviewer = autogen.AssistantAgent(
    name="SEO Reviewer",
    llm_config=llm_config,
    system_message="You are an SEO reviewer, known for "
        "your ability to optimize content for search engines, "
        "ensuring that it ranks well and attracts organic traffic. " 
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role.",
)


In [11]:
legal_reviewer = autogen.AssistantAgent(
    name="Legal Reviewer",
    llm_config=llm_config,
    system_message="You are a legal reviewer, known for "
        "your ability to ensure that content is legally compliant "
        "and free from any potential legal issues. "
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role.",
)

In [12]:
ethics_reviewer = autogen.AssistantAgent(
    name="Ethics Reviewer",
    llm_config=llm_config,
    system_message="You are an ethics reviewer, known for "
        "your ability to ensure that content is ethically sound "
        "and free from any potential ethical issues. " 
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role. ",
)

In [13]:
meta_reviewer = autogen.AssistantAgent(
    name="Meta Reviewer",
    llm_config=llm_config,
    system_message="You are a meta reviewer, you aggregate and review "
    "the work of other reviewers and give a final suggestion on the content.",
)

#### Orchestrate the nested chats to solve the task

In [14]:
def reflection_message(recipient, messages, sender, config):
    return f'''Review the following content. 
            \n\n {recipient.chat_messages_for_summary(sender)[-1]['content']}'''

review_chats = [
    {
     "recipient": SEO_reviewer, 
     "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'Reviewer': '', 'Review': ''}. Here Reviewer should be your role",},
     "max_turns": 1},
    {
    "recipient": legal_reviewer, "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'Reviewer': '', 'Review': ''}.",},
     "max_turns": 1},
    {"recipient": ethics_reviewer, "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'reviewer': '', 'review': ''}",},
     "max_turns": 1},
     {"recipient": meta_reviewer, 
      "message": "Aggregrate feedback from all reviewers and give final suggestions on the writing.", 
     "max_turns": 1},
]


In [15]:
critic.register_nested_chats(
    review_chats,
    trigger=writer,
)

In [16]:
res = critic.initiate_chat(
    recipient=writer,
    message=task,
    max_turns=2,
    summary_method="last_msg"
)

Critic (to Writer):


        Write a concise but engaging blogpost about
       DeepLearning.AI. Make sure the blogpost is
       within 100 words.
       

--------------------------------------------------------------------------------
Writer (to Critic):

Title: Unleashing the Power of DeepLearning.AI

Dive into the world of AI with DeepLearning.AI! Founded by the renowned Andrew Ng, this platform offers top-notch courses on deep learning, neural networks, and machine learning. Whether you're a beginner or an advanced learner, there's something for everyone. With hands-on projects and expert-led videos, you'll sharpen your AI skills and stay ahead in the tech game. Join the community of learners globally, and unlock exciting opportunities in artificial intelligence. Embrace the future with DeepLearning.AI as your guide! Start your AI journey today.

--------------------------------------------------------------------------------

****************************************************

In [17]:
print(res.summary)

Title: Empower Your AI Journey with DeepLearning.AI

Embark on your AI odyssey with DeepLearning.AI by Andrew Ng! Explore cutting-edge courses in deep learning, neural networks, and machine learning, tailored for all skill levels. From hands-on projects to expert-led videos, elevate your AI proficiency and lead the tech wave. Join a global community of learners to unlock doors to exciting AI opportunities. Transparent pricing and data usage ensure a trustworthy experience. Begin shaping your AI future today with DeepLearning.AI. Optimize your learning path and dive into the world of artificial intelligence!
